# Wine Quality
Here, I will apply different ML algorithms to predict quality of wine.

1) **Normal equation**

The prediction of wine quality by **normal equation** is a **linear regression** task. **Normal equation** is an analytical approach to Linear Regression with a Least Square Cost Function.
We can directly find out the value of θ without using Gradient Descent. Following this approach is an effective and time-saving option when are working with a dataset with small features.
(click on the link to find more <a href="https://www.geeksforgeeks.org/ml-normal-equation-in-linear-regression/"><code>link1</code></a> and <a href="http://mlwiki.org/index.php/Normal_Equation"><code>link2</code></a>

2) **Random forest classifier**

3) **Neural Network**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

How to get data online:
<details>
    The code snippet below is responsible for downloading the dataset for example when running via Google Colab. You can also directly download the file using the link if you work with a local setup (in that case, ignore the the block below with !wget)

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv

In [2]:
# load data from csv file and make a numpy array
data = np.genfromtxt('winequality-white.csv',delimiter=";",skip_header=1)

print("data:", data.shape)

# Prepare for proper training
np.random.shuffle(data) # randomly sort examples

data: (4898, 12)


In [ ]:
# take the first 3000 examples for training
x_train = data[:3000,:11] # all features except last column
y_train = data[:3000,11]  # quality column

# and the remaining examples for testing
x_test = data[3000:,:11]
y_test = data[3000:,11]

print("First example:")
print("Features:", x_train[0]) # [0] refers to the first example
print("Quality:", y_train[0])

In [ ]:
features = ["fixed acidity", "volatile acidity", "citric acid", 
            "residual sugar", "chlorides", "free sulfur dioxide", 
            "total sulfur dioxide", "density", "pH", "sulphates", "alcohol"]

## Data visualization

First we want to understand the data better. 
* Plot (`plt.hist`) the distribution of each of the features for the training data.
* the 2D distribution (either `plt.scatter` or `plt.hist2d`) of each feature versus quality.
* Also calculate the correlation coefficient (`np.corrcoef`) for each feature with quality. Which feature by itself seems most predictive for the quality?


<details>
    <a href="https://realpython.com/python-enumerate/"><code>enumerate</code></a>

In [ ]:
# Loop over all features
for element_index, element in enumerate(features):
    
    # 1D Histogram 
    plt.hist(x_train[:,element_index])
    plt.xlabel(element)
    plt.ylabel("Wines")
    plt.show()
    
    # Scatter plot 1 and 2 show same information in different representations.
    # Scatter Plot 1
    plt.scatter(x_train[:,element_index],y_train, s=1, alpha=0.2)
    plt.xlabel(element)
    plt.ylabel("Quality")
    plt.show()

    # Scatter Plot 2
    plt.hist2d(x_train[:,element_index],y_train, bins=[10, np.arange(0.5, 10.5, 1)])
    plt.xlabel(element)
    plt.ylabel("Quality")
    plt.show()
    
    # Calulate correlation coefficient
    plt.clf()
    print(f"Feature: {element}")
    print(f"Correlation coefficient: {np.corrcoef(x_train[:,element_index],y_train)[0,1]:.3f}") 


##  1) Normal equation

* Calculate the linear regression weights by solving the normal equation: 
     $$ W = (x^T x)^{-1} x^T y$$

<details>
    
    * Numpy provides functions for 
        * matrix multiplication (`np.matmul`), 
        * matrix transposition (`.T`),
        * matrix inversion (`np.linalg.inv`).


In [ ]:
# Calulate weights using train data

w = np.matmul(np.matmul(np.linalg.inv(np.matmul(x_train.T, x_train)), x_train.T),y_train)

print(w)
print(w.shape)

* Use the weights to predict the quality for the test dataset.
    * `y_{predict} = x_{test} w`

In [ ]:
# Evaluate linear regression model 
y_pred = np.matmul(x_test,w)

print(x_test.shape,w.shape,y_pred.shape)
print(x_test[0])
print(y_pred[0])

* To find how the predicted quality is good compared to the true quality of the test data, calculate the correlation coefficient between predicted and true quality and draw a scatter plot.

In [ ]:
print("Correlation coefficient:", np.corrcoef(y_pred,y_test)[0,1])

# Prepare scatter plot
plt.scatter(y_pred,y_test)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

## 2) Random Forest Classifier

In [ ]:
# make pandas DataFrame of data
columns = ["fixed acidity", "volatile acidity", "citric acid", 
            "residual sugar", "chlorides", "free sulfur dioxide", 
            "total sulfur dioxide", "density", "pH", "sulphates", "alcohol", "quality"]
df = pd.DataFrame(data, columns=columns )
df.shape

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

Dataset does not have any null.

## Data visualization

In [ ]:
df.hist(bins=20,figsize=(10,10))
plt.show()

Let's check how the quality of wine change by alcohol.

In [ ]:
plt.figure(figsize=[5,5])
plt.bar(df['quality'],df['alcohol'])
plt.xlabel('quality')
plt.ylabel('alcohol')
plt.show()

In [ ]:
# correlation by visualization
plt.figure(figsize=[20,10])
# plot correlation
sb.heatmap(df.corr(),annot=True)
plt.show()

In [ ]:
df.corr(method='pearson',min_periods=1)

In [ ]:
# features and target
X = df.drop(['quality'],axis=1)

Y = df['quality']
X.columns

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)

X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

X_scaled.head(10)

Split train set into random train and validation subsets.

In [ ]:
from sklearn.model_selection import train_test_split
 
# creating train test splits
X_train, X_valid, Y_train, Y_valid = train_test_split(X_scaled, Y, train_size=0.7, test_size=0.3)

print(f"No. of training examples: {X_train.shape[0]}")
print(f"No. of testing examples: {X_valid.shape[0]}")

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_train, Y_train)

In [ ]:
Y_pre = model.predict(X_valid)

Check the accuracy of the prediction

In [ ]:
from sklearn.metrics import mean_squared_error
MSE = mean_squared_error(Y_valid,Y_pre)
RMSE = np.sqrt(MSE)

print('mean squared error is',MSE)
 
print('............................................')
 
print('root mean squared error is',RMSE)

# 3) Neural network

The goal here is the implementation of a simple neural network with one hidden layer using **gradiant descent**.

Consider
* One hidden layer between input and outpu layers,
    * number of features: n_inputs
    * number of neurons in hidden layer: hidden_nodes
* Activation function,
    * reLu activation function

$$\hat{y}=\mathbf{W}^{\prime} \sigma(\mathbf{W} \vec{x}+\vec{b})+b^{\prime}$$

### Normalization:

<details>
    Before apply the neural network to get better results we normalize the input data.
    We apply normalization manually.

In [ ]:
# mean value of each columns of data array by axis = 0
mean = np.mean(data, axis=0)
print(mean.shape)
# standard deviation of each columns of data array
std = np.std(data, axis = 0)

# Normalized data:
data = (data - mean)/std


# determine train and test sets:
X_train = data[:3000,:11]
y_train = data[:3000,11]

X_test = data[3000:,:11]
y_test = data[3000:,11]

print("First example:")
print("Features:", X_train[0])
print("Quality:", y_train[0])

### Random initialization of weights

<details>
    Initialise weights with suitable random distributions:

In [ ]:
# number of nodes in the hidden layer
hidden_nodes = 50
# number of features
n_inputs = 11

# initialise
W = np.random.randn(hidden_nodes,11)*np.sqrt(2./n_inputs)
b = np.zeros(hidden_nodes)
Wp = np.random.randn(hidden_nodes)*np.sqrt(2./hidden_nodes)
bp = np.zeros(1)

print(W.shape)

build a network
* activation function: reLu
we do not use written reLu function and write our own.
* 

### Activation function

In [ ]:
def relu(x):
    return np.maximum(x,0)

In [ ]:
def nn(x,W,b,Wp,bp):
    return np.dot(Wp,relu(np.dot(W,x)+b))+bp

Update weights using **gradiant descent**

For the **regression** problem the loss function is the **mean squared error** between the prediction and the true label $y$:
$$L=(\hat{y}-y)^{2}$$

Taking the partial derivatives - and diligently the applying chain rule - with respect to the different objects yields:

\begin{aligned}
\frac{\partial L}{\partial b^{\prime}} &=2(\hat{y}-y) \\
\frac{\partial L}{\partial \mathbf{W}_{k}^{\prime}} &=2(\hat{y}-y) \sigma\left(\sum_{i} \mathbf{W}_{i k} x_{i}+b_{k}\right) \\
\frac{\partial L}{\partial b_{k}} &=2(\hat{y}-y) \mathbf{W}_{k}^{\prime} \theta\left(\sum_{i} \mathbf{W}_{i k} x_{i}+b_{k}\right) \\
\frac{\partial L}{\partial \mathbf{W}_{k m}} &=2(\hat{y}-y) \mathbf{W}_{m}^{\prime} \theta\left(\sum_{i} \mathbf{W}_{i m} x_{i}+b_{m}\right) x_{k}
\end{aligned}

Here, $\Theta$ denotes the Heaviside step function.
Now, update the weights and bias via learning rate $\alpha$ by

\begin{aligned}
b^{\prime} &= b^{\prime} - \alpha \frac{\partial L}{\partial b^{\prime}} \\
\mathbf{W}_{k}^{\prime} &= \mathbf{W}_{k}^{\prime} - \alpha \frac{\partial L}{\partial \mathbf{W}_{k}^{\prime}} \\
b_{k} &= b_{k} - \alpha \frac{\partial L}{\partial b_{k}} & \\
\mathbf{W}_{k m} &= \mathbf{W}_{k m} - \alpha \frac{\partial L}{\partial \mathbf{W}_{k m}} &
\end{aligned}


<details>
    See the links
    <a href="https://www.tutorialspoint.com/python/python_basic_operators.htm"><code>python_basic_operators</code></a>
    and
    <a href="https://numpy.org/doc/stable/reference/generated/numpy.outer.html"><code>outer_product_of_two_vectors</code></a>


In [ ]:
# learning rate
lr = 0.00005

def update_weights(x,y,W,b,Wp,bp):
    
    z = nn(x,W,b,Wp,bp)
    
    # Use the formulas derived to calculate the gradient for each of W,b,Wp,bp
    delta_bp = 2 * (z-y)
    delta_Wp = delta_bp * relu(np.dot(W,x)+b)
    delta_b = delta_bp * Wp * np.heaviside(np.dot(W,x)+b,0.5)
    delta_W = delta_bp * np.outer(Wp * np.heaviside(np.dot(W,x)+b,0.5),x)
    
    
    # Update the weights/bias following the rule
    bp -= lr * delta_bp
    Wp -= lr * delta_Wp
    b  -= lr * delta_b
    W  -= lr * delta_W
    
    # return new weights and bias
    return W, b, Wp, bp 

In [ ]:
train_losses = []
test_losses = []

# How many epochs to train
n_epochs = 100

# Loop over the epochs
for ep in range(n_epochs):
        
    # Each epoch is a complete over the training data
    for i in range(X_train.shape[0]):
        
        # pick one example
        x = X_train[i]
        y = y_train[i]

        # use it to update the weights
        W, b, Wp, bp = update_weights(x,y, W, b, Wp, bp)
    
    # Calculate predictions for the full training and testing sample
    y_pred_train = [nn(x,W,b,Wp,bp)[0] for x in X_train]
    y_pred = [nn(x,W,b,Wp,bp)[0] for x in X_test]

    train_loss = sum((y_pred_train-y_train)**2) / y_train.shape[0]
    test_loss = sum((y_pred-y_test)**2) / y_test.shape[0] 
    
    # print some information
    print("Epoch:",ep, "Train Loss:", train_loss, "Test Loss:", test_loss)
    
    # store the losses for later use
    train_losses.append(train_loss)
    test_losses.append(test_loss)

In [ ]:
# After the training:
    
# Prepare scatter plot
y_pred = [nn(x,W,b,Wp,bp)[0] for x in X_test]

# now we need to rescale the output to the correct values
y_pred = (y_pred + mean[11])* std[11]
y_test = (y_test + mean[11])* std[11]
y_pred_train = (y_pred_train + mean[11]) * std[11]
y_train = (y_train + mean[11]) * std[11]


print(f"Best loss: {min(test_losses):.3f}, Final loss: {test_losses[-1]:.3f}")

print("Correlation coefficient:", np.corrcoef(y_pred,y_test)[0,1])
plt.scatter(y_pred_train,y_train)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

# Prepare and loss over time
plt.plot(train_losses[2:],label="train")
plt.plot(test_losses[2:],label="test") # we omit the first data points as the first loss is typically very high which makes it difficult to read the plot. 
plt.legend()
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()


# 4) Neural network by Keras

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
data = np.genfromtxt('winequality-white.csv',delimiter=";",skip_header=1)

print("data:", data.shape)

np.random.shuffle(data)

# Normalize
mean = np.mean(data, axis=0)
std = np.std(data, axis = 0)
print(mean.shape)

data = (data - mean)/std

X_train = data[:3000,:11]
y_train = data[:3000,11] 

X_test = data[3000:,:11]
y_test = data[3000:,11]

print("First example:")
print("Features:", X_train.shape)
print("Quality:", y_train.shape)

In [ ]:
hidden_nodes = 50
input = X_train.shape[(1)]

In [ ]:
model = keras.Sequential([
                          layers.Dense(units=hidden_nodes, activation='relu', input_shape=[input]),
])

In [ ]:
model.compile(
    optimizer="adam",
    loss="mae",
)

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=256,
    epochs=100,
)

In [ ]:
import pandas as pd

# convert the training history to a dataframe
history_df = pd.DataFrame(history.history)
# use Pandas native plot method
history_df['loss'].plot();
history_df['val_loss'].plot();